In [1]:
import pandas as pd
from neo4j import GraphDatabase

In [2]:
tablas_names = ['estados', 'productos', 'tiendas']
tablas2_name = ['clientes']
relaciones_n = ['localizada_en', 'vende_en', 'compra']

In [3]:
tablas = []
for idx, elem in enumerate(tablas_names):
    elem = pd.read_csv(f'./{tablas_names[idx]}.csv', encoding='latin1')
    tablas.append(elem)

In [4]:
tablas2 = []
for idx, elem in enumerate(tablas2_name):
    elem = pd.read_csv(f'./{tablas2_name[idx]}.csv', encoding='latin1')
    tablas2.append(elem)

In [5]:
relaciones = []
for idx, elem in enumerate(relaciones_n):
    elem = pd.read_csv(f'./{relaciones_n[idx]}.csv', encoding='latin1')
    relaciones.append(elem)

In [6]:
#tablas
#tablas2
#relaciones

In [7]:
#Tablas con 1 columna
for idx, df in enumerate(tablas2):
    
    nombre_tabla = tablas2_name[idx]
    id_column = df.iloc[:,0]
    execution_commands = []

    for i in range(id_column.size):
        neo4j_create_statement = f"create ({nombre_tabla[0]}:{nombre_tabla} {{{id_column.name}:'{id_column.iloc[i]}'}})"
        execution_commands.append(neo4j_create_statement)


    def execute_transactions(execution_commands):
        data_base_connection = GraphDatabase.driver(uri = "bolt://localhost:7687", auth=("neo4j", "neo5j"))
        session = data_base_connection.session()    
        for i in execution_commands:
            session.run(i)
    execute_transactions(execution_commands)

In [8]:
#Tablas de 2 columnas
for idx, df in enumerate(tablas):

    nombre_tabla = tablas_names[idx]
    id_column = df.iloc[:,1]
    data_column = df.iloc[:,0]
    execution_commands = []

    for i in range(id_column.size):
        neo4j_create_statement = f"create ({nombre_tabla[0]}:{nombre_tabla} {{{id_column.name}:'{id_column.iloc[i]}', {data_column.name}: '{data_column.iloc[i]}'}})"
        execution_commands.append(neo4j_create_statement)


    def execute_transactions(execution_commands):
        data_base_connection = GraphDatabase.driver(uri = "bolt://localhost:7687", auth=("neo4j", "neo5j"))
        session = data_base_connection.session()    
        for i in execution_commands:
            session.run(i)


    execute_transactions(execution_commands)    
    

## Relaciones

In [9]:
#Localizada en:

idx = 0
df = relaciones[idx]

nombre_tabla = relaciones_n[idx]
pid_column = df.iloc[:,1]
sid_column = df.iloc[:,0]

execution_commands = []
for i in range(relaciones[0].shape[0]):
    neo4j_create_statement = f"match (t:tiendas {{{pid_column.name}: '{pid_column.iloc[i]}'}}) match (e:estados {{{sid_column.name}: '{sid_column.iloc[i]}'}}) create (t)-[lc:{nombre_tabla}]->(e)"
    execution_commands.append(neo4j_create_statement)

def execute_transactions(execution_commands):
    data_base_connection = GraphDatabase.driver(uri = "bolt://localhost:7687", auth=("neo4j", "neo5j"))
    session = data_base_connection.session()    
    for i in execution_commands:
        session.run(i)


execute_transactions(execution_commands)  

#f"match t:tiendas {{{pid_column.name}: {pid_column_data}}} match e:estados {{{sid_column.name}: {sid_column_data}}} create (t)-[lc:{nombre_tabla}]->(e)"


In [10]:
#Vende en:
idx = 1
df = relaciones[idx]

nombre_tabla = relaciones_n[idx]
pid_column = df.iloc[:,1]
sid_column = df.iloc[:,0]
data_column = df.iloc[:,2]

execution_commands = []
for i in range(pid_column.size):
    neo4j_create_statement = f"match (t:tiendas {{{pid_column.name}: '{pid_column.iloc[i]}'}}) match (p:productos {{{sid_column.name}: '{sid_column.iloc[i]}'}}) create (p)-[ve:{nombre_tabla}]->(t) set ve.incumplimiento = '{data_column[i]}'"
    execution_commands.append(neo4j_create_statement)


def execute_transactions(execution_commands):
    data_base_connection = GraphDatabase.driver(uri = "bolt://localhost:7687", auth=("neo4j", "neo5j"))
    session = data_base_connection.session()    
    for i in execution_commands:
        session.run(i)

#len(execution_commands)
execute_transactions(execution_commands)   

In [11]:
idx = 2
df = relaciones[idx]
df
#relaciones_n[idx]

,Id_Cliente,Id_Producto,Id_Tienda
0,c0,133p,2432t
1,c0,13p,2450t
2,c0,134p,3616t
3,c0,13p,2433t
4,c0,20p,4837t
...,...,...,...
994,c99,3p,6660t
995,c99,2p,6988t
996,c99,6p,7783t
997,c99,47p,107t


In [12]:
#Compra:
idx = 2
df = relaciones[idx]

nombre_tabla = relaciones_n[idx]
col_clie = df.Id_Cliente
col_prod = df.Id_Producto
col_tien = df.Id_Tienda

execution_commands = []
for i in range(col_clie.size):
    neo4j_create_statement = f"match (c:clientes {{{col_clie.name}: '{col_clie.iloc[i]}'}}) match (p:productos {{{col_prod.name}: '{col_prod.iloc[i]}'}}) create (c)-[ve:{nombre_tabla}]->(p) set ve.en = '{col_tien[i]}'"
    execution_commands.append(neo4j_create_statement)


def execute_transactions(execution_commands):
    data_base_connection = GraphDatabase.driver(uri = "bolt://localhost:7687", auth=("neo4j", "neo5j"))
    session = data_base_connection.session()    
    for i in execution_commands:
        session.run(i)

#execution_commands[:15]
execute_transactions(execution_commands)   

#f"match (c:clientes {{{col_clie.name}: '{col_clie.iloc[0]}'}}) match (p:productos {{{col_prod.name}: '{col_prod.iloc[0]}'}}) create (p)-[co:{nombre_tabla}]->(t) set co.en = '{col_tien[0]}'"


In [13]:
MATCH (order:Order {orderID: row.OrderID})
MATCH (product:Product {productID: row.ProductID})
MERGE (order)-[op:CONTAINS]->(product)

SyntaxError: invalid syntax (1461673234.py, line 1)

In [ ]:
tiendas -> localizada_en -> estados

productos -> vende_en -> tiendas

clientes -> compra -> productos